In [26]:
import cv2
import mediapipe as mp
import numpy as np

In [27]:
mp_drawing = mp.solutions.drawing_utils   # type: ignore
mp_drawing_styles = mp.solutions.drawing_styles  # type: ignore
mp_pose = mp.solutions.pose  # type: ignore

In [28]:
# For webcam input:
cap = cv2.VideoCapture("datasets/chute01/cam1.avi")

In [29]:
#Variables
Title = "Fall Detection"

In [30]:
#Pose Detection Model variables
with mp_pose.Pose(
   min_detection_confidence=0.5,
   min_tracking_confidence=0.5,
   static_image_mode=False,
   model_complexity=2,
   smooth_landmarks=True,
   enable_segmentation=True,
   smooth_segmentation=True
) as pose:
  
  #Loop to read the video
  while cap.isOpened():
    success, video = cap.read()

    #if the there is no video or frames being played, it will close the window
    if not success:
        cv2.destroyWindow(Title)
        cv2.destroyWindow("Person")
        break

    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    (regions, _) = hog.detectMultiScale(video,
                                        winStride=(4, 4),
                                        padding=(4, 4),
                                        scale=1.05)
    
    #removing the background apart from the person
    for (x, y, w, h) in regions:
        #cut the person out of the video
        test = video[y:y+h, x:x+w]
    # Draw the pose annotation on the video.
    video.flags.writeable = True
    results = pose.process(video)
    mp_drawing.draw_landmarks(
        video,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    

    
    def fall_detection():

        if results.pose_landmarks is not None:
            if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y > 0.9 or results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y > 0.9:
                print("Fall Detected")
                cv2.putText(video, "Fall Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    fall_detection()

    cv2.imshow(Title, video)
    cv2.imshow("Person", test) # type: ignore

    if cv2.waitKey(5) & 0xFF == 27:
        cv2.destroyWindow(Title)
        break